# Cassava Leaf Disease Modelling
There is already a great kernel [Vision Transformer (ViT): Tutorial + Baseline](https://www.kaggle.com/abhinand05/vision-transformer-vit-tutorial-baseline) that shows us how to use visiontransformer on TPU and why. Here, we make further improvements on the basis of the official implementation, in order to provide better pre-training parameters and user-friendly API as `Effecientnet-PyTorch`. You can find all the details on https://github.com/tczhangzhi/VisionTransformer-Pytorch.


In [ ]:
# !pip install vision_transformer_pytorch

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')
import timm


In [ ]:
# Import libraries
import os
import pandas as pd
import albumentations as albu
import matplotlib.pyplot as plt
import json
import seaborn as sns
import cv2
import albumentations as albu
import numpy as np

# Visualization

# Modelling

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split
from albumentations.pytorch import ToTensorV2
# from efficientnet_pytorch import EfficientNet
import time
import datetime
import copy

In [ ]:
# DataSet class

class CassavaDataset(Dataset):
    def __init__(self,df:pd.DataFrame,imfolder:str,train:bool = True, transforms=None):
        self.df=df
        self.imfolder=imfolder
        self.train=train
        self.transforms=transforms
        
    def __getitem__(self,index):
        im_path=os.path.join(self.imfolder,self.df.iloc[index]['image_id'])
        x=cv2.imread(im_path,cv2.IMREAD_COLOR)
        x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
        
        if(self.transforms):
            x=self.transforms(image=x)['image']
        
        if(self.train):
            y=self.df.iloc[index]['label']
            return x,y
        else:
            return x
        
    def __len__(self):
        return len(self.df)

In [ ]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        print(self.model)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:

# model_name = 'efficientnet-b7'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model=models.(pretrained=True)
# model.fc=nn.Linear(512,5)
# model = EfficientNet.from_pretrained(model_name, num_classes=5) 
# model=models.resnext50_32x4d()#Add Pretrained=True to use pretrained with internet enabled
# model.fc=nn.Linear(model.fc.in_features,5)
#model = VisionTransformer.from_name('ViT-B_16', num_classes=5) 
#model = CassvaImgClassifier(5)

In [ ]:
model = CassvaImgClassifier('vit_large_patch16_384', n_class=5)
model.to(device)

In [ ]:
!ls ../input/vitweights

Train the model after uncommenting below

In [ ]:
# trained_model=train_model(datasets,dataloaders,model,criterion,optimizer,scheduler,num_epochs,device)

In [ ]:
# Epoch 0/5
# ----------
# train Loss: 0.5318 Acc: 0.8119
# valid Loss: 0.4009 Acc: 0.8650

# Epoch 1/5
# ----------
# train Loss: 0.4384 Acc: 0.8467
# valid Loss: 0.3999 Acc: 0.8612

# Epoch 2/5
# ----------
# train Loss: 0.3511 Acc: 0.8778
# valid Loss: 0.3558 Acc: 0.8771

# Epoch 3/5
# ----------
# train Loss: 0.3266 Acc: 0.8879
# valid Loss: 0.3468 Acc: 0.8836

# Epoch 4/5
# ----------
# train Loss: 0.3066 Acc: 0.8924
# valid Loss: 0.3384 Acc: 0.8911

# Epoch 5/5
# ----------
# train Loss: 0.3060 Acc: 0.8926
# valid Loss: 0.3421 Acc: 0.8869

# Training complete in 121m 52s
# Best val Acc: 0.891121

Save the model after training

Load the model when model is trained and saved and notebook has to be run without internet

In [ ]:
model.load_state_dict(torch.load('../input/zzkvit/vit_large_patch16_384_best_fold.pth'))

# Submission

In [ ]:
test_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
image_path = "../input/cassava-leaf-disease-classification/test_images/"
# fake targets
test_targets = test_df.label.values


test_aug = albu.Compose([
            albu.CenterCrop(512, 512, p=1.),
            albu.Resize(384, 384),
            albu.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0),
            ToTensorV2()], p=1.)

test_dataset=CassavaDataset(
    df=test_df,
    imfolder=image_path,
    train=False,
    transforms=test_aug
)

test_loader =  DataLoader(
        test_dataset,
        batch_size=4,
        num_workers=4,
        shuffle=False,
)

In [ ]:
predictions=[]

for imgs in test_loader:

    imgs = imgs.to(device)
    with torch.no_grad():
        model=model.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        predicted=predicted.to('cpu')
        predictions.append(predicted)


In [ ]:
test_df['label'] = np.concatenate(predictions)


In [ ]:
test_df.to_csv('submission.csv', index=False)

In [ ]:
!ls ../input/vitbaseline